In [1]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

## TAXI

In [26]:
start_date = '2022-09-01'
end_date = '2022-09-30'

In [27]:
taxi = f"""

WITH base_month_customer AS 
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d') AS month,
    customer_id,
    --MAX(taxi_lifetime_last_ride_date) 
    taxi_lifetime_last_ride_date,
    taxi_lifetime_first_ride_date,
    --MAX(taxi_lifetime_rides) 
    taxi_lifetime_rides

FROM datasets.iallocator_customer_segments
WHERE 
    run_date BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date)) , '%Y-%m-%d') 
    AND run_date = DATE_FORMAT(DATE_TRUNC('month', date(run_date)), '%Y-%m-%d')
    AND (taxi_lifetime_rides > 0 or taxi_lifetime_rides is not null or taxi_lifetime_last_ride_date is not null)
--GROUP BY 1,2
),

base_month_segment AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    customer_id,
    CASE
    WHEN taxi_lifetime_rides IS NULL OR DATE(month) = DATE(taxi_lifetime_first_ride_date) THEN 'LTR=0'
    WHEN taxi_lifetime_rides BETWEEN 1 AND 4 THEN '1-4'
    WHEN (taxi_lifetime_rides BETWEEN 5 AND 20) AND (taxi_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '5-20'
    WHEN (taxi_lifetime_rides BETWEEN 21 AND 50) AND (taxi_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '21-50'
    WHEN (taxi_lifetime_rides > 50) AND (taxi_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY)) THEN '50+'
    WHEN taxi_lifetime_rides >= 5 THEN '>=5'
    ELSE 'LTR=0'
    END ltr_taxi,
    CASE 
    WHEN taxi_lifetime_last_ride_date IS NULL OR DATE(month) = DATE(taxi_lifetime_first_ride_date) THEN 'LTR=0'
    WHEN taxi_lifetime_last_ride_date >= (DATE_TRUNC('month', DATE(month)) - INTERVAL '30' DAY) THEN '1. LAST_30_DAYS'
    WHEN taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '90' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '31' DAY) THEN '2. 31_90_DAYS'
    WHEN taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '180' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '91' DAY) THEN '3. 91_180_DAYS'
    WHEN taxi_lifetime_last_ride_date BETWEEN (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) AND (DATE_TRUNC('month', DATE(month)) - INTERVAL '181' DAY) THEN '4. 181_365_DAYS'
    WHEN taxi_lifetime_last_ride_date < (DATE_TRUNC('month', DATE(month)) - INTERVAL '365' DAY) THEN '5. 365_ABOVE'
    ELSE 'LTR=0'
    END recency_taxi
FROM base_month_customer
),

base_month_total AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(month)) , '%Y-%m-%d') AS month,
    COUNT(DISTINCT customer_id) existing_custr_total 
FROM base_month_customer
GROUP BY 1
),

customer_rf_daily_kpi AS
(
SELECT 
    day,
    customerid,
    rr_sessions_unique_daily,
    net_rides_daily,
    ao_sessions_unique_daily,
    fe_sessions_unique_daily,
    discount_daily,
    subtotal_daily
        
FROM datasets.customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d')
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
    AND service_name IN ('Link', 'Auto')
    AND customerid IS NOT NULL
),

rr_net AS
(
SELECT 
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customerid as customer_id,
    SUM(rr_sessions_unique_daily) rr,
    SUM(net_rides_daily) net_rides,
    SUM(discount_daily) burn,
    SUM(subtotal_daily) subtotal
FROM customer_rf_daily_kpi
WHERE 
    day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
    AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
GROUP BY 1,2
),

max_daily_ao AS
(
SELECT 
        day,
        customer_id,
        SUM(ao) AS ao,
        SUM(fe) AS fe
FROM
    (
    SELECT 
        customerid as customer_id,
        day,
        max(ao_sessions_unique_daily) AS ao,
        max(fe_sessions_unique_daily) AS fe
    FROM datasets.customer_rf_daily_kpi
    WHERE 
        day BETWEEN DATE_FORMAT(DATE_TRUNC('month', cast('{start_date}' as date)), '%Y-%m-%d') 
        AND DATE_FORMAT(DATE_TRUNC('month', cast('{end_date}' as date) + INTERVAL '28' DAY) - INTERVAL '1' DAY, '%Y-%m-%d')
        AND customerid IS NOT NULL
    GROUP BY 1,2
    )
GROUP BY 1,2
),

monthly_ao AS
(
SELECT  
    DATE_FORMAT(DATE_TRUNC('month', DATE(day)), '%Y-%m-%d') AS month,
    customer_id,
    SUM(ao) AS ao,
    SUM(fe) AS fe
FROM max_daily_ao
GROUP BY 1,2
),

base_current AS (
SELECT 
        COALESCE(b.month,c.month,r.month) month,
        -- b.month,
        COALESCE(b.customer_id,c.customer_id,r.customer_id) customers,
        b.customer_id bcustomer_id,
        COALESCE(b.ltr_taxi,'NEW') ltr_taxi,
        COALESCE(b.recency_taxi, 'NEW') recency_taxi,
        c.customer_id acustomer_id,
        r.customer_id rcustomer_id,
        CASE 
        WHEN c.ao = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN c.ao = 0 AND c.fe > 0 THEN c.fe
        WHEN c.ao = 0 AND r.rr > 0 THEN r.rr
        WHEN c.ao IS NULL THEN 0
        ELSE c.ao END ao,
        
        CASE
        WHEN c.fe = 0 AND r.net_rides > 0 THEN r.net_rides 
        WHEN c.fe = 0 AND r.rr > 0 THEN r.rr
        WHEN c.fe IS NULL THEN 0
        ELSE c.fe END fe,
        
        CASE
        WHEN r.rr = 0 AND r.net_rides > 0 THEN r.net_rides
        WHEN r.rr IS NULL THEN 0
        ELSE r.rr END rr,
        
        COALESCE(r.net_rides, 0) net_rides,
        COALESCE(r.burn, 0) burn,
        COALESCE(r.subtotal, 0) subtotal

FROM base_month_segment AS b
FULL JOIN monthly_ao AS c ON b.customer_id = c.customer_id AND b.month = c.month 
FULL JOIN rr_net AS r ON b.customer_id = r.customer_id AND c.month = r.month
),

detailed_view AS 
(
SELECT 
        c.month,
        ltr_taxi,
        case 
        when ltr_taxi = '1-4' then 1
        when ltr_taxi = '5-20' then 2
        when ltr_taxi = '21-50' then 3
        when ltr_taxi = '50+' then 4
        when ltr_taxi = '>=5' then 5
        when ltr_taxi = 'LTR=0' then 6
        when ltr_taxi = 'NEW' then 7
        end row_order,
        recency_taxi,
        COUNT(DISTINCT bcustomer_id) existing_custr_count,
        COUNT(DISTINCT CASE WHEN ao > 0 THEN acustomer_id END) ao_users,
        COUNT(DISTINCT CASE WHEN fe > 0 THEN acustomer_id END) fe_users,
        COUNT(DISTINCT CASE WHEN rr > 0 THEN rcustomer_id END) rr_users,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN rcustomer_id END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
        
FROM base_current c
GROUP BY 1,2,3,4
),

summary_view AS (
SELECT 
        c.month,
        COUNT(DISTINCT CASE WHEN net_rides > 0 THEN c.customers END) net_users,
        SUM(net_rides) net_rides,
        SUM(burn) discount,
        SUM(subtotal) subtotal
FROM base_current c
GROUP BY 1
),

a as 
(
SELECT  
        a.month,
        a.row_order,
        a.ltr_taxi,
        a.recency_taxi,
        a.existing_custr_count,
        COALESCE(try(a.existing_custr_count*100.00/b.existing_custr_total), 0) existing_custr_distr,
        a.ao_users,
        COALESCE(try(a.ao_users*100.00/a.existing_custr_count), 0) ao_conversion, 
        a.fe_users,
        COALESCE(try(a.fe_users*100.00/a.existing_custr_count), 0) fe_conversion, 
        a.rr_users,
        COALESCE(try(a.rr_users*100.00/a.existing_custr_count), 0) rr_conversion,
        COALESCE(try(a.rr_users*100.00/a.fe_users), 0) fe_rr,
        a.net_users,
        COALESCE(try(a.net_users*100.00/a.existing_custr_count), 0) net_conversion,
        COALESCE(try(a.net_users*100.00/a.rr_users), 0) rr_net,
        COALESCE(try(a.net_users*100.00/c.net_users), 0) net_users_distr,
        a.net_rides,
        COALESCE(try(a.net_rides*100.00/c.net_rides), 0) net_rides_distr,
        COALESCE(try(a.net_rides*1.00/a.net_users), 0) rpc,
        a.discount,
        a.subtotal
        
FROM detailed_view a
LEFT JOIN base_month_total b ON a.month = b.month
LEFT JOIN summary_view c ON a.month = c.month
ORDER BY 1,2,3
),

b as 
(
SELECT
        month,
        8 row_order,
        'TOTAL' ltr_taxi,
        '' recency_taxi,
        SUM(existing_custr_count) existing_custr_count,
        100.00 existing_custr_distr,
        SUM(ao_users) ao_users,
        COALESCE(try(SUM(ao_users)*100.00/SUM(existing_custr_count)), 0) ao_conversion,
        SUM(fe_users) fe_users,
        COALESCE(try(SUM(fe_users)*100.00/SUM(existing_custr_count)), 0) fe_conversion,
        SUM(rr_users) rr_users,
        COALESCE(try(SUM(rr_users)*100.00/SUM(existing_custr_count)), 0) rr_conversion,
        COALESCE(try(SUM(rr_users)*100.00/SUM(fe_users)), 0) fe_rr,
        SUM(net_users) net_users,
        COALESCE(try(SUM(net_users)*100.00/SUM(existing_custr_count)), 0) net_conversion,
        COALESCE(try(SUM(net_users)*100.00/SUM(rr_users)), 0) rr_net,
        100.00 net_users_distr,
        SUM(net_rides) net_rides,
        100.00 net_rides_distr,
        COALESCE(try(SUM(net_rides)*1.0/SUM(net_users)), 0) rpc,
        SUM(discount) discount,
        SUM(subtotal) subtotal
FROM a
GROUP BY 1
),

final as 
(
select * from a
union
select * from b
)
select 
        month,
        ltr_taxi,recency_taxi,
        existing_custr_count,existing_custr_distr,
        ao_users,ao_conversion,
        fe_users,fe_conversion,
        rr_users,rr_conversion,fe_rr "fe_rr(users)",
        net_users,net_conversion, rr_net "rr_net(users)", net_users_distr,
        net_rides,net_rides_distr,
        rpc,
        discount,
        subtotal,
        COALESCE(TRY(discount*100.0/subtotal),0) discount_perc,
        COALESCE(TRY(discount*1.0/net_rides),0) dpr
from final
order by 1,row_order,3

"""

In [28]:
df_taxi = pd.read_sql(taxi, conn)

In [29]:
df_taxi_sep = df_taxi
df_taxi_sep.head()

,month,ltr_taxi,recency_taxi,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-09-01,1-4,1. LAST_30_DAYS,2092063,6.92,1269773,60.69,1107439,52.94,858445,...,32.12,78.29,10.61,1757315,7.64,2.61,9103330.0,160444379.0,5.673823,5.180249
1,2022-09-01,1-4,2. 31_90_DAYS,2502549,8.28,888697,35.51,710434,28.39,477262,...,13.29,69.69,5.25,636426,2.77,1.91,3740130.0,59654669.0,6.269635,5.876771
2,2022-09-01,1-4,3. 91_180_DAYS,2324552,7.69,491303,21.14,382603,16.46,240330,...,7.04,68.11,2.58,297727,1.29,1.82,1905205.0,26918627.0,7.077646,6.399168
3,2022-09-01,1-4,4. 181_365_DAYS,2126507,7.03,251511,11.83,203190,9.56,125944,...,4.03,68.13,1.35,161099,0.70,1.88,1070747.0,13908517.0,7.698499,6.646515
4,2022-09-01,1-4,5. 365_ABOVE,5258034,17.39,249819,4.75,208082,3.96,132445,...,1.79,71.16,1.49,192036,0.83,2.04,1363501.0,16292842.0,8.368712,7.100236


In [19]:
df_taxi_nov

,month,ltr_taxi,recency_taxi,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-11-01,1-4,1. LAST_30_DAYS,1978327,5.93,1166841,58.98,1011483,51.13,776717,...,30.03,76.49,9.25,1523841,6.55,2.56,5991290.0,1.328951e+08,4.508286,3.931703
1,2022-11-01,1-4,2. 31_90_DAYS,2608528,7.82,911649,34.95,735516,28.20,495795,...,12.94,68.07,5.25,640089,2.75,1.90,2883590.0,5.757350e+07,5.008537,4.504983
2,2022-11-01,1-4,3. 91_180_DAYS,2792799,8.37,581630,20.83,452774,16.21,283531,...,6.70,65.95,2.91,334908,1.44,1.79,1651142.0,2.965174e+07,5.568450,4.930136
3,2022-11-01,1-4,4. 181_365_DAYS,2693290,8.07,319080,11.85,253924,9.43,154311,...,3.82,66.63,1.60,185466,0.80,1.80,1009692.0,1.566130e+07,6.447051,5.444081
4,2022-11-01,1-4,5. 365_ABOVE,5563172,16.67,251994,4.53,206416,3.71,129942,...,1.61,68.85,1.39,174145,0.75,1.95,991528.0,1.416097e+07,7.001835,5.693692
5,2022-11-01,5-20,1. LAST_30_DAYS,2373880,7.11,1871742,78.85,1755207,73.94,1518823,...,53.72,83.96,19.85,4388148,18.85,3.44,11955704.0,3.685783e+08,3.243735,2.724544
6,2022-11-01,21-50,1. LAST_30_DAYS,1185961,3.55,1035439,87.31,997882,84.14,920278,...,69.67,89.78,12.86,4183185,17.97,5.06,10565616.0,3.312180e+08,3.189928,2.525735
7,2022-11-01,50+,1. LAST_30_DAYS,857265,2.57,788777,92.01,771280,89.97,739804,...,81.35,94.27,10.86,6734903,28.94,9.66,20541632.0,4.766242e+08,4.309817,3.050026
8,2022-11-01,>=5,2. 31_90_DAYS,2607108,7.81,1419098,54.43,1258340,48.27,965330,...,28.05,75.77,11.39,1719034,7.39,2.35,5028414.0,1.431457e+08,3.512794,2.925139
9,2022-11-01,>=5,3. 91_180_DAYS,1758563,5.27,564331,32.09,473789,26.94,328081,...,13.32,71.38,3.65,480368,2.06,2.05,1565787.0,3.903910e+07,4.010818,3.259557


In [25]:
df_taxi_oct

,month,ltr_taxi,recency_taxi,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-10-01,1-4,1. LAST_30_DAYS,2054883,6.46,1260253,61.33,1112033,54.12,865732,...,31.56,74.91,10.03,1592931,7.28,2.46,6496863.0,1.469782e+08,4.420290,4.078559
1,2022-10-01,1-4,2. 31_90_DAYS,2537361,7.98,933573,36.79,764311,30.12,520962,...,13.40,65.27,5.26,618047,2.82,1.82,2905194.0,5.954570e+07,4.878932,4.700604
2,2022-10-01,1-4,3. 91_180_DAYS,2606147,8.20,569033,21.83,450703,17.29,286531,...,6.89,62.62,2.77,308287,1.41,1.72,1589677.0,2.884357e+07,5.511374,5.156484
3,2022-10-01,1-4,4. 181_365_DAYS,2369857,7.45,292957,12.36,237634,10.03,146832,...,3.90,62.98,1.43,161766,0.74,1.75,894236.0,1.435991e+07,6.227310,5.527960
4,2022-10-01,1-4,5. 365_ABOVE,5404842,17.00,258101,4.78,214435,3.97,135998,...,1.65,65.67,1.38,168370,0.77,1.89,968963.0,1.432515e+07,6.764070,5.754962
5,2022-10-01,5-20,1. LAST_30_DAYS,2409696,7.58,1940766,80.54,1838798,76.31,1603441,...,55.38,83.23,20.63,4420196,20.20,3.31,12139351.0,3.916806e+08,3.099299,2.746338
6,2022-10-01,21-50,1. LAST_30_DAYS,1160823,3.65,1024023,88.22,992455,85.50,919651,...,70.72,89.27,12.69,3968075,18.14,4.83,10062569.0,3.309350e+08,3.040648,2.535882
7,2022-10-01,50+,1. LAST_30_DAYS,808983,2.54,747103,92.35,733004,90.61,703641,...,81.50,93.70,10.19,5913968,27.03,8.97,18198848.0,4.441552e+08,4.097408,3.077265
8,2022-10-01,>=5,2. 31_90_DAYS,2421764,7.62,1338589,55.27,1195441,49.36,920308,...,27.68,72.84,10.36,1459788,6.67,2.18,4289961.0,1.298823e+08,3.302960,2.938756
9,2022-10-01,>=5,3. 91_180_DAYS,1565811,4.92,519382,33.17,441487,28.20,308145,...,13.42,68.18,3.25,404524,1.85,1.93,1344956.0,3.489238e+07,3.854584,3.324787


In [30]:
df_taxi_sep

,month,ltr_taxi,recency_taxi,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-09-01,1-4,1. LAST_30_DAYS,2092063,6.92,1269773,60.69,1107439,52.94,858445,...,32.12,78.29,10.61,1757315,7.64,2.61,9103330.0,1.604444e+08,5.673823,5.180249
1,2022-09-01,1-4,2. 31_90_DAYS,2502549,8.28,888697,35.51,710434,28.39,477262,...,13.29,69.69,5.25,636426,2.77,1.91,3740130.0,5.965467e+07,6.269635,5.876771
2,2022-09-01,1-4,3. 91_180_DAYS,2324552,7.69,491303,21.14,382603,16.46,240330,...,7.04,68.11,2.58,297727,1.29,1.82,1905205.0,2.691863e+07,7.077646,6.399168
3,2022-09-01,1-4,4. 181_365_DAYS,2126507,7.03,251511,11.83,203190,9.56,125944,...,4.03,68.13,1.35,161099,0.70,1.88,1070747.0,1.390852e+07,7.698499,6.646515
4,2022-09-01,1-4,5. 365_ABOVE,5258034,17.39,249819,4.75,208082,3.96,132445,...,1.79,71.16,1.49,192036,0.83,2.04,1363501.0,1.629284e+07,8.368712,7.100236
5,2022-09-01,5-20,1. LAST_30_DAYS,2377055,7.86,1898544,79.87,1781032,74.93,1540393,...,55.34,85.39,20.77,4708062,20.47,3.58,17049404.0,4.107484e+08,4.150815,3.621321
6,2022-09-01,21-50,1. LAST_30_DAYS,1102385,3.65,966384,87.66,930857,84.44,859347,...,70.76,90.77,12.32,4168544,18.12,5.34,13877881.0,3.414786e+08,4.064056,3.329191
7,2022-09-01,50+,1. LAST_30_DAYS,743524,2.46,683417,91.92,667393,89.76,639453,...,81.32,94.55,9.55,5909943,25.69,9.78,21722196.0,4.414891e+08,4.920212,3.675534
8,2022-09-01,>=5,2. 31_90_DAYS,2263123,7.49,1208787,53.41,1058209,46.76,804184,...,27.20,76.55,9.72,1423472,6.19,2.31,5850838.0,1.236832e+08,4.730504,4.110259
9,2022-09-01,>=5,3. 91_180_DAYS,1363881,4.51,440169,32.27,368273,27.00,253048,...,13.50,72.75,2.91,377987,1.64,2.05,1718955.0,3.152469e+07,5.452726,4.547656


In [32]:
frames = [df_taxi_nov, df_taxi_oct, df_taxi_sep]
nov_oct_sep = pd.concat(frames)
nov_oct_sep

,month,ltr_taxi,recency_taxi,existing_custr_count,existing_custr_distr,ao_users,ao_conversion,fe_users,fe_conversion,rr_users,...,net_conversion,rr_net(users),net_users_distr,net_rides,net_rides_distr,rpc,discount,subtotal,discount_perc,dpr
0,2022-11-01,1-4,1. LAST_30_DAYS,1978327,5.93,1166841,58.98,1011483,51.13,776717,...,30.03,76.49,9.25,1523841,6.55,2.56,5991290.0,1.328951e+08,4.508286,3.931703
1,2022-11-01,1-4,2. 31_90_DAYS,2608528,7.82,911649,34.95,735516,28.20,495795,...,12.94,68.07,5.25,640089,2.75,1.90,2883590.0,5.757350e+07,5.008537,4.504983
2,2022-11-01,1-4,3. 91_180_DAYS,2792799,8.37,581630,20.83,452774,16.21,283531,...,6.70,65.95,2.91,334908,1.44,1.79,1651142.0,2.965174e+07,5.568450,4.930136
3,2022-11-01,1-4,4. 181_365_DAYS,2693290,8.07,319080,11.85,253924,9.43,154311,...,3.82,66.63,1.60,185466,0.80,1.80,1009692.0,1.566130e+07,6.447051,5.444081
4,2022-11-01,1-4,5. 365_ABOVE,5563172,16.67,251994,4.53,206416,3.71,129942,...,1.61,68.85,1.39,174145,0.75,1.95,991528.0,1.416097e+07,7.001835,5.693692
5,2022-11-01,5-20,1. LAST_30_DAYS,2373880,7.11,1871742,78.85,1755207,73.94,1518823,...,53.72,83.96,19.85,4388148,18.85,3.44,11955704.0,3.685783e+08,3.243735,2.724544
6,2022-11-01,21-50,1. LAST_30_DAYS,1185961,3.55,1035439,87.31,997882,84.14,920278,...,69.67,89.78,12.86,4183185,17.97,5.06,10565616.0,3.312180e+08,3.189928,2.525735
7,2022-11-01,50+,1. LAST_30_DAYS,857265,2.57,788777,92.01,771280,89.97,739804,...,81.35,94.27,10.86,6734903,28.94,9.66,20541632.0,4.766242e+08,4.309817,3.050026
8,2022-11-01,>=5,2. 31_90_DAYS,2607108,7.81,1419098,54.43,1258340,48.27,965330,...,28.05,75.77,11.39,1719034,7.39,2.35,5028414.0,1.431457e+08,3.512794,2.925139
9,2022-11-01,>=5,3. 91_180_DAYS,1758563,5.27,564331,32.09,473789,26.94,328081,...,13.32,71.38,3.65,480368,2.06,2.05,1565787.0,3.903910e+07,4.010818,3.259557


In [34]:
nov_oct_sep.to_csv("taxi_nov_oct_sep.csv", index = False)